# **Hilal Properties**

### **Setting The Environment**

In [ ]:
import csv
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

### **Opening the Hilal Properties page using Chrome**

In [105]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

BASE_URL = "https://hilalprp.com.om"
URL_TEMPLATE = BASE_URL + "/properties-search/page/{}/?status=for-rent"
MAX_PAGES = 18

# Data storage
hilal_data = {
    'property_title': [],
    'location': [],
    'number_of_rooms': [],
    'price': [],
    'size': [],
    'listing_type': []
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# Start scraping
for current_page in range(1, MAX_PAGES + 1):
    url = URL_TEMPLATE.format(current_page)
    print(f"\nFetching page {current_page}: {url}")

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch page {current_page}: {e}")
        break

    soup = BeautifulSoup(response.content, 'html.parser')
    listings = soup.find_all('article', class_='rh_list_card')

    if not listings:
        print(f"No listings found on page {current_page}. Stopping.")
        break

    print(f"{len(listings)} listings found on page {current_page}")

    for listing in listings:
        # Title
        title_tag = listing.find("h3")
        title = title_tag.get_text(strip=True) if title_tag else None

        # Price
        price_tag = listing.find('p', class_='price')
        price = price_tag.text.strip() if price_tag else None

        # Listing Type (e.g., For Rent)
        listing_type_tag = listing.find('span', class_='status')
        listing_type = listing_type_tag.text.strip() if listing_type_tag else "N/A"

        # Number of Rooms and Size
        figure_tags = listing.find_all('span', class_='figure')
        number_of_rooms = figure_tags[0].text.strip() if len(figure_tags) >= 1 else None

        size = None
        for fig in figure_tags:
            label_tag = fig.find_next_sibling('span', class_='label')
            if label_tag and 'sqmt' in label_tag.text.lower():
                size = fig.text.strip()
                break

        # Property link
        link_tag = listing.find('a', href=True)
        listing_url = link_tag['href'] if link_tag else None

        # Location (from property detail page)
        location = None
        if listing_url:
            full_url = listing_url if listing_url.startswith("http") else BASE_URL + listing_url
            try:
                detail_res = requests.get(full_url, headers=headers)
                detail_res.raise_for_status()
                detail_soup = BeautifulSoup(detail_res.content, 'html.parser')
                location_tag = detail_soup.find('a', href=lambda href: href and '/property-city/' in href)
                location = location_tag.text.strip() if location_tag else None
            except Exception as e:
                print(f"Failed to fetch detail page: {e}")

        # Append to data
        hilal_data['property_title'].append(title)
        hilal_data['location'].append(location)
        hilal_data['number_of_rooms'].append(number_of_rooms)
        hilal_data['price'].append(price)
        hilal_data['size'].append(size)
        hilal_data['listing_type'].append(listing_type)

    time.sleep(1)  # Be polite to the server

print("\nScraping complete.")

# Create DataFrame
df_hilal = pd.DataFrame(hilal_data)
print(df_hilal.head(10))


Fetching page 1: https://hilalprp.com.om/properties-search/page/1/?status=for-rent
10 listings found on page 1

Fetching page 2: https://hilalprp.com.om/properties-search/page/2/?status=for-rent
10 listings found on page 2

Fetching page 3: https://hilalprp.com.om/properties-search/page/3/?status=for-rent
10 listings found on page 3

Fetching page 4: https://hilalprp.com.om/properties-search/page/4/?status=for-rent
10 listings found on page 4

Fetching page 5: https://hilalprp.com.om/properties-search/page/5/?status=for-rent
10 listings found on page 5

Fetching page 6: https://hilalprp.com.om/properties-search/page/6/?status=for-rent
10 listings found on page 6

Fetching page 7: https://hilalprp.com.om/properties-search/page/7/?status=for-rent
10 listings found on page 7

Fetching page 8: https://hilalprp.com.om/properties-search/page/8/?status=for-rent
10 listings found on page 8

Fetching page 9: https://hilalprp.com.om/properties-search/page/9/?status=for-rent
10 listings found on

In [106]:
# Save to CSV
df_hilal.to_csv("hilal_properties_for_rent.csv", index=False)
print(f"\nSaved {len(df_hilal)} listings to 'hilal_properties_for_rent.csv'")


Saved 173 listings to 'hilal_properties_for_rent.csv'
